In [5]:
import json
import torch.nn as nn

with open('unique_OA_Pairs.json','r') as f:
    UniqeOAPairs = json.load(f)

UniqeOAPairs = [tuple(pair) for pair in UniqeOAPairs]
num_oas = len(UniqeOAPairs) 

with open('OA_Pairs.json','r') as f:
    id2OAPairs = json.load(f)

id2OAPairs = {id: list(set(tuple(pair) for pair in id2OAPairs[id])) for id in id2OAPairs.keys()}

with open('vcoco_processed_data_all_2014.json', 'r') as f:
    vcoco_data = json.load(f)
    
ids = list(vcoco_data.keys())

train_ids = ids[:8000]
test_ids = ids[8000:10000]

train_data = [vcoco_data[id] for id in train_ids]
test_data = [vcoco_data[id] for id in test_ids]


import torch
id2OAOneHot = {}

for id, pairs in id2OAPairs.items():
    # Filter pairs that are in UniqeOAPairs
    valid_pairs = [pair for pair in pairs if pair in UniqeOAPairs]

    # Get indices of valid pairs
    indices = [UniqeOAPairs.index(pair) for pair in valid_pairs]

    # Convert indices to one-hot encoding
    oneHot = nn.functional.one_hot(torch.tensor(indices, dtype=int), num_classes=len(UniqeOAPairs))#gives batch of hot encoding (len(indices), len(UNiqeOAPairs))
    
    oneHot = oneHot.sum(axis = 0) #converts to single vector (1, len(UNiqeOAPairs))
    # Store the one-hot encodings for the ID
    id2OAOneHot[id] = oneHot
